In [1]:
# To display full output in Notebook, instead of only the last result
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.interpolate import Akima1DInterpolator
from statsmodels.tsa.seasonal import STL

In [3]:
# Load and view weather data
weather_full = pd.read_excel("BMD_data_full.xlsx", engine="openpyxl")
weather_full.head(5)

,location,date,rainfall,temp_max,temp_min,rH_min,rH_max,wind_speed,wind_direction
0,Bogura,2020-10-22,0.0,29.6,25.5,76.0,93.0,5.6,South/South-easterly
1,Bogura,2020-10-23,41.0,25.0,24.5,93.0,96.0,3.7,South/South-easterly
2,Bogura,2020-10-24,7.0,25.5,23.4,94.0,96.0,3.7,South/South-easterly
3,Bogura,2020-10-25,0.0,30.0,23.6,71.0,96.0,1.9,South/South-easterly
4,Bogura,2020-10-26,0.0,32.2,24.3,60.0,95.0,1.9,South/South-easterly


In [4]:
# Create season variable
def assign_season(date: pd.Timestamp) -> str:
    """
    Map a date to a custom seasonal label such as '2021S1', '2021S2', etc.

    Season boundaries (inclusive):
        S1: 22 Oct (previous year) – 25 Jan (current year)
        S2: 26 Jan – 25 Apr
        S3: 26 Apr – 31 Jul
        S4: 01 Aug – 21 Oct

    Notes
    -----
    * 22 Oct belongs to S1 of the *next* calendar year (e.g. 2020-10-22 → 2021S1).
    * 21 Oct belongs to S4 of the *same* calendar year.
    """
    m, d, y = date.month, date.day, date.year

    # S1 (cross-year period)
    if (m == 10 and d >= 22) or m in (11, 12):
        return f"{y + 1}S1"
    if m == 1 and d <= 25:
        return f"{y}S1"

    # S2
    if (m == 1 and d >= 26) or m in (2, 3) or (m == 4 and d <= 25):
        return f"{y}S2"

    # S3
    if (m == 4 and d >= 26) or m in (5, 6) or (m == 7 and d <= 31):
        return f"{y}S3"

    # S4
    if m == 8 or m == 9 or (m == 10 and d <= 21):
        return f"{y}S4"

    # Any date outside the defined windows (unlikely) → NaN
    return pd.NA

# Add the new 'season' column to your DataFrame
weather_full["season"] = weather_full["date"].apply(assign_season)

In [5]:
# Quick sanity check
print(weather_full["season"].value_counts(dropna=False).sort_index())

season
2021S1    288
2021S2    270
2021S3    291
2021S4    246
2022S1    288
2022S2    270
2022S3    291
2022S4    246
Name: count, dtype: int64


In [6]:
weather_full.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2190 entries, 0 to 2189
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   location        2190 non-null   object        
 1   date            2190 non-null   datetime64[ns]
 2   rainfall        2145 non-null   float64       
 3   temp_max        2113 non-null   float64       
 4   temp_min        2139 non-null   float64       
 5   rH_min          2130 non-null   float64       
 6   rH_max          2130 non-null   float64       
 7   wind_speed      2130 non-null   float64       
 8   wind_direction  2190 non-null   object        
 9   season          2190 non-null   object        
dtypes: datetime64[ns](1), float64(6), object(3)
memory usage: 171.2+ KB


In [7]:
# Check rows with null values
pd.set_option('display.max_rows', None)
missing_rows = weather_full[weather_full.isna().any(axis=1)]
missing_rows
pd.reset_option('display.max_rows')

,location,date,rainfall,temp_max,temp_min,rH_min,rH_max,wind_speed,wind_direction,season
33,Bogura,2020-11-24,0.0,NaN,15.5,NaN,NaN,NaN,North/North-westerly,2021S1
46,Bogura,2020-12-07,0.0,NaN,17.0,NaN,NaN,NaN,North/North-westerly,2021S1
47,Bogura,2020-12-08,NaN,NaN,NaN,NaN,NaN,NaN,North/North-westerly,2021S1
59,Bogura,2020-12-20,0.0,NaN,10.8,NaN,NaN,NaN,North/North-westerly,2021S1
60,Bogura,2020-12-21,NaN,NaN,NaN,NaN,NaN,NaN,North/North-westerly,2021S1
61,Bogura,2020-12-22,NaN,NaN,NaN,NaN,NaN,NaN,North/North-westerly,2021S1
73,Bogura,2021-01-03,0.0,NaN,12.8,NaN,NaN,NaN,North/North-westerly,2021S1
74,Bogura,2021-01-04,NaN,NaN,NaN,NaN,NaN,NaN,North/North-westerly,2021S1
75,Bogura,2021-01-05,NaN,NaN,NaN,NaN,NaN,NaN,North/North-westerly,2021S1
132,Bogura,2021-03-03,NaN,NaN,NaN,NaN,NaN,NaN,North/North-westerly,2021S2


# Fill in null values

## temp_max, temp_min, rH_min, rH_max, wind_speed
Akima interpolation + 3-day centered rolling mean

In [10]:
# Step 1: Rename columns
weather_full.rename(columns={
    'temp_max': 'temp_max_raw',
    'temp_min': 'temp_min_raw',
    'rH_max': 'rH_max_raw',
    'rH_min': 'rH_min_raw',
    'wind_speed': 'wind_speed_raw'
}, inplace=True)

In [11]:
# Step 2: Create boolean NA indicator columns
for var in ["temp_max_raw", "temp_min_raw", "rH_max_raw", "rH_min_raw", "wind_speed_raw"]:
    weather_full[f"{var}_na"] = weather_full[var].isna()

In [12]:
# Step 3: Akima interpolation for each location separately
def akima_fill(series: pd.Series) -> pd.Series:
    """
    Fill NA gaps in a 1-D time-indexed series using Akima interpolation.
    Non-missing values remain unchanged.
    """
    x = np.arange(len(series))
    mask = series.notna()                      # existing observations
    if mask.sum() < 3:                         # Akima needs ≥3 points
        return series                          # leave as is
    akima = Akima1DInterpolator(x[mask], series[mask])
    filled = series.copy()
    filled[~mask] = akima(x[~mask])
    return filled

In [13]:
for var in ["temp_max_raw", "temp_min_raw", "rH_max_raw", "rH_min_raw", "wind_speed_raw"]:
    weather_full[var] = (
        weather_full
        .groupby("location", group_keys=False)[var]
        .apply(akima_fill)
    )

In [14]:
# Step 4: Three-day centered rolling mean (smoothing)
for var in ["temp_max_raw", "temp_min_raw", "rH_max_raw", "rH_min_raw", "wind_speed_raw"]:
    smooth_col = var.replace("_raw", "_smooth")
    weather_full[smooth_col] = (
        weather_full
        .groupby("location", group_keys=False)[var]
        .transform(lambda s: s.rolling(window=3, center=True, min_periods=1).mean())
    )

In [15]:
# Step 5: Create the analysis columns, choosing raw vs. smoothed per original NA flag
for base in ["temp_max", "temp_min", "rH_max", "rH_min", "wind_speed"]:
    raw_col     = f"{base}_raw"
    smooth_col  = f"{base}_smooth"
    na_flag_col = f"{raw_col}_na"
    weather_full[base] = np.where(
        weather_full[na_flag_col],      # True  → original was NA
        weather_full[smooth_col],       #        → use smoothed value
        weather_full[raw_col]           # False → keep original observation
    )

In [16]:
# Drop temporary smooth columns
weather_full = weather_full.drop(columns=["temp_max_smooth", "temp_min_smooth", 
                                          "rH_max_smooth", "rH_min_smooth", 
                                          "wind_speed_smooth"])

In [17]:
# Verify results
pd.set_option('display.max_columns', None)
print(weather_full.head(5))
pd.reset_option('display.max_columns')

  location       date  rainfall  temp_max_raw  temp_min_raw  rH_min_raw  \
0   Bogura 2020-10-22       0.0          29.6          25.5        76.0   
1   Bogura 2020-10-23      41.0          25.0          24.5        93.0   
2   Bogura 2020-10-24       7.0          25.5          23.4        94.0   
3   Bogura 2020-10-25       0.0          30.0          23.6        71.0   
4   Bogura 2020-10-26       0.0          32.2          24.3        60.0   

   rH_max_raw  wind_speed_raw        wind_direction  season  temp_max_raw_na  \
0        93.0             5.6  South/South-easterly  2021S1            False   
1        96.0             3.7  South/South-easterly  2021S1            False   
2        96.0             3.7  South/South-easterly  2021S1            False   
3        96.0             1.9  South/South-easterly  2021S1            False   
4        95.0             1.9  South/South-easterly  2021S1            False   

   temp_min_raw_na  rH_max_raw_na  rH_min_raw_na  wind_speed_raw_na 

In [18]:
print("\nMissing values after processing:")
print(weather_full[['temp_max', 'temp_min', 'rH_max', 'rH_min', 'wind_speed']].isna().sum())


Missing values after processing:
temp_max      0
temp_min      0
rH_max        0
rH_min        0
wind_speed    0
dtype: int64


In [19]:
# Export the processed DataFrame to Excel (optional, for inspection only)
weather_full.to_excel('weather_full_v1.xlsx', index=False)

## rainfall

Rainfall gap-filling with STL decomposition + 14-day seasonal window matching
* Adds `rainfall_na`  (True if original value was NaN)
* Replaces NaNs in `rainfall` with imputed values
  based on a ±7-day window whose seasonal component
  is phase-aligned within ±0.5 mm, plus a random residual.

In [21]:
# Setup
np.random.seed(114)
weather_full = weather_full.sort_values(
    ["location", "date"]
).reset_index(drop=True)

In [22]:
# Flag rows originally missing
weather_full["rainfall_na"] = weather_full["rainfall"].isna()

In [23]:
def impute_rainfall_stl(group: pd.DataFrame) -> pd.DataFrame:
    """
    STL-based rainfall imputation with non-negative enforcement, 
    weighted seasonal matching, and rounding of imputed values.
    """
    g = group.copy().sort_values('date')
    
    # Pre-clean existing negative values
    g['rainfall'] = g['rainfall'].clip(lower=0)
    
    # Temporary linear interpolation (non-negative)
    y = g['rainfall'].interpolate('linear', limit_direction='both').clip(lower=0)
    
    # STL decomposition with robust fitting
    stl = STL(y, period=365, robust=True)
    res = stl.fit()
    
    # Store components
    g['rain_trend'] = res.trend
    g['rain_seasonal'] = res.seasonal
    g['rain_residual'] = res.resid
    
    # Identify missing indices
    na_idx = g.index[g['rainfall_na']]
    
    for idx in na_idx:
        target_date = g.at[idx, 'date']
        target_seasonal = g.at[idx, 'rain_seasonal']
        
        # ±7-day window
        mask = (
            g['date'].between(
                target_date - pd.Timedelta(days=7), 
                target_date + pd.Timedelta(days=7)
            ) & 
            (~g['rainfall_na']) & 
            (g['date'] != target_date))
        
        # Seasonal matching (0.5mm threshold)
        seasonal_diff = g['rain_seasonal'].sub(target_seasonal).abs()
        candidates = g.loc[mask & (seasonal_diff < 0.5)]
        
        if not candidates.empty:
            # Weighted average by seasonal similarity
            weights = 1 / (seasonal_diff[candidates.index] + 0.01)
            base = np.average(candidates['rainfall'], weights=weights)
            
            # Only use positive residuals
            positive_resids = candidates['rain_residual'][candidates['rain_residual'] > 0]
            noise = np.random.choice(positive_resids) if len(positive_resids) > 0 else 0
            
            # ROUND TO 1 DECIMAL PLACE
            filled_value = max(base + noise, 0)
            g.at[idx, 'rainfall'] = round(filled_value, 1)
        else:
            # ROUND TO 1 DECIMAL PLACE
            filled_value = max(g.at[idx, 'rain_trend'] + g.at[idx, 'rain_seasonal'], 0)
            g.at[idx, 'rainfall'] = round(filled_value, 1)
    
    # Final non-negative enforcement (doesn't affect original values)
    g.loc[g['rainfall_na'], 'rainfall'] = g.loc[g['rainfall_na'], 'rainfall'].clip(lower=0)
    return g.drop(columns=['rain_trend', 'rain_seasonal', 'rain_residual'])


In [24]:
# Execute imputation
weather_full = (
    weather_full
    .sort_values(['location', 'date'])
    .groupby('location', group_keys=False)
    .apply(impute_rainfall_stl)
)

/var/folders/qx/5zbpywt5661d6xr06v91bdyr0000gn/T/ipykernel_36799/1648372106.py:6: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(impute_rainfall_stl)


In [25]:
# Verify results
print("Remaining NaNs in rainfall:", weather_full["rainfall"].isna().sum())
print("Negative values after imputation:", (weather_full['rainfall'] < 0).sum())
print("Filled values statistics:")
print(weather_full.loc[weather_full['rainfall_na'], 'rainfall'].describe())

Remaining NaNs in rainfall: 0
Negative values after imputation: 0
Filled values statistics:
count    45.000000
mean      0.802222
std       3.678778
min       0.000000
25%       0.000000
50%       0.000000
75%       0.000000
max      24.000000
Name: rainfall, dtype: float64


In [26]:
pd.set_option('display.max_columns', None)
print(weather_full.head(5))
pd.reset_option('display.max_columns')

  location       date  rainfall  temp_max_raw  temp_min_raw  rH_min_raw  \
0   Bogura 2020-10-22       0.0          29.6          25.5        76.0   
1   Bogura 2020-10-23      41.0          25.0          24.5        93.0   
2   Bogura 2020-10-24       7.0          25.5          23.4        94.0   
3   Bogura 2020-10-25       0.0          30.0          23.6        71.0   
4   Bogura 2020-10-26       0.0          32.2          24.3        60.0   

   rH_max_raw  wind_speed_raw        wind_direction  season  temp_max_raw_na  \
0        93.0             5.6  South/South-easterly  2021S1            False   
1        96.0             3.7  South/South-easterly  2021S1            False   
2        96.0             3.7  South/South-easterly  2021S1            False   
3        96.0             1.9  South/South-easterly  2021S1            False   
4        95.0             1.9  South/South-easterly  2021S1            False   

   temp_min_raw_na  rH_max_raw_na  rH_min_raw_na  wind_speed_raw_na 

In [27]:
# Export the processed DataFrame to Excel (for inspection only)
weather_full.to_excel('weather_full_v2.xlsx', index=False)

# Weather Feature Engineering

## Temperature

In [30]:
# Create daily temperature range column
weather_full['temp_range'] = weather_full['temp_max'] - weather_full['temp_min']

In [31]:
# Initialize summary dataframe with unique location-season combinations
weather_summary = weather_full[['location', 'season']].drop_duplicates().reset_index(drop=True)

In [32]:
# Group by location and season
grouped = weather_full.groupby(['location', 'season'])

In [33]:
# Define aggregation functions
def percentile(n):
    def percentile_(x):
        return np.nanpercentile(x, n)
    percentile_.__name__ = 'p%s' % n
    return percentile_

In [34]:
# Temperature Max Statistics
temp_max_stats = grouped['temp_max'].agg([
    ('temp_max_mean', 'mean'),
    ('temp_max_median', 'median'),
    ('temp_max_std', 'std'),
    ('temp_max_95p', percentile(95)),
    ('temp_max_above_30_days', lambda x: (x > 30).sum()),
    ('temp_max_above_35_days', lambda x: (x > 35).sum()),
    ('total_days', 'count')
])

temp_max_stats['temp_max_above_30_pct'] = temp_max_stats['temp_max_above_30_days'] / temp_max_stats['total_days']
temp_max_stats['temp_max_above_35_pct'] = temp_max_stats['temp_max_above_35_days'] / temp_max_stats['total_days']

In [35]:
# Temperature Min Statistics
temp_min_stats = grouped['temp_min'].agg([
    ('temp_min_mean', 'mean'),
    ('temp_min_median', 'median'),
    ('temp_min_std', 'std'),
    ('temp_min_5p', percentile(5)),
    ('temp_min_below_10_days', lambda x: (x < 10).sum()),
    ('temp_min_below_5_days', lambda x: (x < 5).sum()),
    ('total_days', 'count')
])

temp_min_stats['temp_min_below_10_pct'] = temp_min_stats['temp_min_below_10_days'] / temp_min_stats['total_days']
temp_min_stats['temp_min_below_5_pct'] = temp_min_stats['temp_min_below_5_days'] / temp_min_stats['total_days']

In [36]:
# Temperature Range Statistics
temp_range_stats = grouped['temp_range'].agg([
    ('temp_range_mean', 'mean'),
    ('temp_range_median', 'median'),
    ('temp_range_std', 'std'),
    ('temp_range_above_10_days', lambda x: (x > 10).sum()),
    ('temp_range_above_15_days', lambda x: (x > 15).sum()),
    ('temp_range_above_20_days', lambda x: (x > 20).sum()),
    ('total_days', 'count')
])

temp_range_stats['temp_range_above_10_pct'] = temp_range_stats['temp_range_above_10_days'] / temp_range_stats['total_days']
temp_range_stats['temp_range_above_15_pct'] = temp_range_stats['temp_range_above_15_days'] / temp_range_stats['total_days']
temp_range_stats['temp_range_above_20_pct'] = temp_range_stats['temp_range_above_20_days'] / temp_range_stats['total_days']

In [37]:
# Merge all statistics into the summary dataframe
weather_summary = weather_summary.merge(temp_max_stats, on=['location', 'season'])
weather_summary = weather_summary.drop(columns=['temp_max_above_30_days', 'temp_max_above_35_days', 'total_days']) 

In [38]:
weather_summary = weather_summary.merge(temp_min_stats, on=['location', 'season'])
weather_summary = weather_summary.drop(columns=['temp_min_below_10_days', 'temp_min_below_5_days', 'total_days']) 

In [39]:
weather_summary = weather_summary.merge(temp_range_stats, on=['location', 'season'])
weather_summary = weather_summary.drop(columns=['temp_range_above_10_days', 'temp_range_above_15_days', 
                                                'temp_range_above_20_days', 'total_days']) 

In [40]:
# Display the first few rows
pd.set_option('display.max_columns', None)
print(weather_summary.head())
pd.reset_option('display.max_columns')

  location  season  temp_max_mean  temp_max_median  temp_max_std  \
0   Bogura  2021S1      27.183567            26.80      4.028707   
1   Bogura  2021S2      31.418161            31.90      3.996879   
2   Bogura  2021S3      33.192784            33.50      2.225573   
3   Bogura  2021S4      33.364791            33.55      1.865792   
4   Bogura  2022S1      27.667708            27.50      3.217861   

   temp_max_95p  temp_max_above_30_pct  temp_max_above_35_pct  temp_min_mean  \
0        33.000               0.229167               0.010417      16.145773   
1        36.555               0.666667               0.188889      19.289380   
2        36.140               0.917526               0.195876      25.731959   
3        36.000               0.939024               0.170732      26.369609   
4        32.400               0.302083               0.000000      16.148958   

   temp_min_median  temp_min_std  temp_min_5p  temp_min_below_10_pct  \
0             15.7      4.207620      

## Growing Degree Days (GDD)

In [42]:
# Calculate Growing Degree Days (GDD) using (Tmax + Tmin)/2 - Tbase
# Where T_base is set 10°C
T_BASE = 10
weather_full['daily_gdd'] = (
    (weather_full['temp_max'] + weather_full['temp_min']) / 2 - T_BASE
).clip(lower=0)

# Calculate seasonal GDD statistics
gdd_stats = weather_full.groupby(['location', 'season']).agg(
    gdd_total=('daily_gdd', 'sum'),
    gdd_mean_daily=('daily_gdd', 'mean'),
    gdd_days_above_zero=('daily_gdd', lambda x: (x > 0).sum()),
    total_days=('date', 'count')
).reset_index()

# Calculate percentage of biologically active days
gdd_stats['gdd_days_above_zero_pct'] = gdd_stats['gdd_days_above_zero'] / gdd_stats['total_days']

In [43]:
# Add to weather_summary
gdd_cols = ['gdd_total', 'gdd_mean_daily', 'gdd_days_above_zero_pct']
weather_summary = weather_summary.merge(
    gdd_stats[['location', 'season'] + gdd_cols],
    on=['location', 'season']
)

In [44]:
# Verify new columns
print(weather_summary[['location', 'season', 'gdd_total', 'gdd_mean_daily', 'gdd_days_above_zero_pct']].head())

  location  season    gdd_total  gdd_mean_daily  gdd_days_above_zero_pct
0   Bogura  2021S1  1119.808315       11.664670                      1.0
1   Bogura  2021S2  1381.839341       15.353770                      1.0
2   Bogura  2021S3  1887.850000       19.462371                      1.0
3   Bogura  2021S4  1629.110425       19.867200                      1.0
4   Bogura  2022S1  1143.200000       11.908333                      1.0


In [45]:
print(weather_summary['gdd_days_above_zero_pct'].tolist())

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]


In [46]:
# Drop the column with zero variation
weather_summary = weather_summary.drop(columns='gdd_days_above_zero_pct')

## Relative Humidity (rH)

In [48]:
# Calculate daily humidity range
weather_full['rh_range'] = weather_full['rH_max'] - weather_full['rH_min']

In [49]:
# Calculate basic statistics
humidity_stats = weather_full.groupby(['location', 'season']).agg(
    rh_max_mean=('rH_max', 'mean'),
    rh_max_std=('rH_max', 'std'),
    rh_min_mean=('rH_min', 'mean'),
    rh_min_std=('rH_min', 'std'),
    rh_range_mean=('rh_range', 'mean'),
    rh_range_std=('rh_range', 'std'),
    total_days=('date', 'count')
).reset_index()

# Calculate high/low humidity days
humidity_stats['high_humidity_days'] = weather_full.groupby(['location', 'season']).apply(
    lambda x: (x['rH_max'] > 85).sum()
).reset_index(drop=True)

humidity_stats['low_humidity_days'] = weather_full.groupby(['location', 'season']).apply(
    lambda x: (x['rH_min'] < 30).sum()
).reset_index(drop=True)

# Calculate percentages
humidity_stats['high_humidity_pct'] = humidity_stats['high_humidity_days'] / humidity_stats['total_days']
humidity_stats['low_humidity_pct'] = humidity_stats['low_humidity_days'] / humidity_stats['total_days']

/var/folders/qx/5zbpywt5661d6xr06v91bdyr0000gn/T/ipykernel_36799/2257772212.py:13: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  humidity_stats['high_humidity_days'] = weather_full.groupby(['location', 'season']).apply(
/var/folders/qx/5zbpywt5661d6xr06v91bdyr0000gn/T/ipykernel_36799/2257772212.py:17: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  humidity_stats['low_humidity_days'] = weather_full.groupby(['

In [50]:
# Select necessary columns for merge
humidity_cols = [
    'location', 'season', 'rh_max_mean', 'rh_max_std', 
    'rh_min_mean', 'rh_min_std', 'rh_range_mean', 'rh_range_std',
    'high_humidity_pct', 'low_humidity_pct'
]

# Merge with weather_summary
weather_summary = weather_summary.merge(
    humidity_stats[humidity_cols], 
    on=['location', 'season']
)

## Vapor Pressure Deficit (VPD)

In [52]:
# Calculate VPD features
weather_full['vpd_upper'] = 0.61078 * np.exp(
    17.269 * weather_full['temp_max'] / (weather_full['temp_max'] + 237.3)
) * (1 - weather_full['rH_min']/100)

weather_full['vpd_lower'] = 0.61078 * np.exp(
    17.269 * weather_full['temp_min'] / (weather_full['temp_min'] + 237.3)
) * (1 - weather_full['rH_max']/100)

weather_full['vpd_middle'] = (weather_full['vpd_upper'] + weather_full['vpd_lower']) / 2

weather_full['water_stress_day'] = weather_full['vpd_upper'] > 1.5

weather_full['night_recovery_risk'] = weather_full['vpd_lower'] > 0.8

# Add to weather_summary
vpd_stats = weather_full.groupby(['location','season']).agg(
    vpd_upper_mean=('vpd_upper', 'mean'),
    vpd_lower_mean=('vpd_lower', 'mean'),
    vpd_middle_mean=('vpd_middle', 'mean'),
    vpd_stressed=('water_stress_day', 'sum'),
    vpd_recovery_risk=('night_recovery_risk', 'sum'),
    total_days=('date', 'count')
)

weather_summary = weather_summary.merge(vpd_stats, on=['location', 'season'])
weather_summary['vpd_stressed_pct'] = weather_summary['vpd_stressed'] / weather_summary['total_days']
weather_summary['vpd_recovery_risk_pct'] = weather_summary['vpd_recovery_risk'] / weather_summary['total_days']

In [53]:
# Drop temperary column
weather_summary = weather_summary.drop(columns=['total_days', 'vpd_stressed', 'vpd_recovery_risk'])

In [54]:
# Verify results
print(weather_summary[['location', 'season', 'vpd_upper_mean', 'vpd_lower_mean', 'vpd_stressed_pct', 'vpd_recovery_risk_pct']].head())

  location  season  vpd_upper_mean  vpd_lower_mean  vpd_stressed_pct  \
0   Bogura  2021S1        1.870738        0.137968          0.697917   
1   Bogura  2021S2        2.977061        0.367861          0.911111   
2   Bogura  2021S3        1.882334        0.270762          0.701031   
3   Bogura  2021S4        1.818585        0.241409          0.731707   
4   Bogura  2022S1        1.959907        0.172245          0.770833   

   vpd_recovery_risk_pct  
0               0.000000  
1               0.066667  
2               0.010309  
3               0.000000  
4               0.000000  


## Rainfall

In [56]:
# Calculate consecutive dry days within each location
def calculate_dry_days(group):
    """Calculate consecutive dry days within a location (cross-season)"""
    g = group.sort_values('date')
    
    # Initialize dry_days column
    g['dry_days'] = 0
    
    # Track current dry streak
    current_streak = 0
    
    for i, row in g.iterrows():
        if row['rainfall'] > 0:
            current_streak = 0  # Reset on rainy day
        else:
            current_streak += 1  # Increment on dry day
        g.at[i, 'dry_days'] = current_streak
    
    return g

# Apply to each location separately
weather_full = weather_full.groupby('location', group_keys=False).apply(calculate_dry_days)

/var/folders/qx/5zbpywt5661d6xr06v91bdyr0000gn/T/ipykernel_36799/3424156729.py:22: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  weather_full = weather_full.groupby('location', group_keys=False).apply(calculate_dry_days)


In [57]:
# Calculate rainfall statistics by location-season
rainfall_stats = weather_full.groupby(['location', 'season']).agg(
    total_rainfall=('rainfall', 'sum'),
    rainfall_mean=('rainfall', 'mean'),
    heavy_rain_days=('rainfall', lambda x: (x > 20).sum()),
    rainy_days=('rainfall', lambda x: (x > 0).sum()),
    max_consecutive_no_rain_days=('dry_days', 'max'),
    total_days=('date', 'count')
).reset_index()

In [58]:
# Calculate rainfall intensity index
rainfall_stats['rain_intensity_index'] = rainfall_stats['total_rainfall'] / rainfall_stats['rainy_days']

# Handle cases with zero rainy days (avoid division by zero)
rainfall_stats.loc[rainfall_stats['rainy_days'] == 0, 'rain_intensity_index'] = 0

In [59]:
# Calculate percentage of rainy days and heavy rain days
rainfall_stats['rainy_days_pct'] = rainfall_stats['rainy_days'] / rainfall_stats['total_days']
rainfall_stats['heavy_rain_days_pct'] = rainfall_stats['heavy_rain_days'] / rainfall_stats['total_days']

In [60]:
# Merge results with weather_summary
rainfall_cols = ['total_rainfall', 'rainfall_mean', 'rainy_days_pct', 'heavy_rain_days_pct', 
                 'max_consecutive_no_rain_days', 'rain_intensity_index']

weather_summary = weather_summary.merge(
    rainfall_stats[['location', 'season'] + rainfall_cols],
    on=['location', 'season']
)

In [61]:
# Verify results
print(weather_summary[['location', 'season', 'total_rainfall', 'heavy_rain_days_pct', 
                       'rainy_days_pct', 'max_consecutive_no_rain_days', 
                       'rain_intensity_index']].head())

  location  season  total_rainfall  heavy_rain_days_pct  rainy_days_pct  \
0   Bogura  2021S1            48.0             0.010417        0.020833   
1   Bogura  2021S2            20.0             0.000000        0.044444   
2   Bogura  2021S3           963.0             0.195876        0.577320   
3   Bogura  2021S4           671.5             0.158537        0.524390   
4   Bogura  2022S1            24.0             0.000000        0.052083   

   max_consecutive_no_rain_days  rain_intensity_index  
0                            93             24.000000  
1                           139              5.000000  
2                             8             17.196429  
3                             5             15.616279  
4                            45              4.800000  


## Temperature-Humidity Index（THI）

In [63]:
# Calculate THI
weather_full['mean_temp'] = (weather_full['temp_max'] + weather_full['temp_min']) / 2
weather_full['mean_rh'] = (weather_full['rH_max'] + weather_full['rH_min']) / 2
weather_full['thi'] = 0.8 * weather_full['mean_temp'] + (weather_full['mean_rh'] / 100) * (weather_full['mean_temp'] - 14.4) + 46.4

In [64]:
# Classify heat stress
def classify_thi(thi):
    if thi < 72: return 'None'
    elif 72 <= thi < 80: return 'Mild'
    elif 80 <= thi < 90: return 'Moderate'
    else: return 'Severe'

weather_full['thi_class'] = weather_full['thi'].apply(classify_thi)

In [65]:
# Create complete location-season grid
all_combinations = pd.MultiIndex.from_product(
    [weather_full['location'].unique(), weather_full['season'].unique()],
    names=['location', 'season']
).to_frame(index=False)

In [66]:
# Calculate base statistics
base_stats = weather_full.groupby(['location', 'season']).agg(
    thi_mean=('thi', 'mean'),
    thi_std=('thi', 'std'),
    thi_95p=('thi', percentile(95)),
    thi_5p=('thi', percentile(5)),
    total_days=('date', 'count')
).reset_index()

In [67]:
# Calculate stress days with complete coverage
stress_levels = ['Mild', 'Moderate', 'Severe']
stress_counts = []

for level in stress_levels:
    level_df = weather_full[weather_full['thi_class'] == level].groupby(
        ['location', 'season']
    ).size().reset_index(name=f'thi_{level.lower()}_days')
    
    # Merge with all combinations to ensure complete coverage
    level_full = all_combinations.merge(
        level_df, 
        on=['location', 'season'], 
        how='left'
    ).fillna({f'thi_{level.lower()}_days': 0})
    
    stress_counts.append(level_full)

# Combine all stress counts
thi_stats = base_stats
for df in stress_counts:
    thi_stats = thi_stats.merge(df, on=['location', 'season'])


In [68]:
# Calculate percentages with zero protection
thi_stats['thi_mild_pct'] = thi_stats['thi_mild_days'] / thi_stats['total_days']
thi_stats['thi_moderate_pct'] = thi_stats['thi_moderate_days'] / thi_stats['total_days']
thi_stats['thi_severe_pct'] = thi_stats['thi_severe_days'] / thi_stats['total_days']

# Calculate combined high stress (≥80 THI)
thi_stats['thi_high_stress_days'] = thi_stats['thi_moderate_days'] + thi_stats['thi_severe_days']
thi_stats['thi_high_stress_pct'] = thi_stats['thi_high_stress_days'] / thi_stats['total_days']

In [69]:
# Merge with weather_summary
thi_cols = [
    'thi_mean', 'thi_std', 'thi_95p', 'thi_5p',
    'thi_mild_pct', 'thi_moderate_pct',
    'thi_severe_pct', 'thi_high_stress_pct'
]

weather_summary = weather_summary.merge(
    thi_stats[['location', 'season'] + thi_cols],
    on=['location', 'season'],
    how='left'  # Ensure no rows are dropped
)

# Fill any remaining NaNs
for col in thi_cols:
    if col.startswith('thi_') and col.endswith('_days'):
        weather_summary[col] = weather_summary[col].fillna(0)
    elif col.startswith('pct_'):
        weather_summary[col] = weather_summary[col].fillna(0.0)

In [70]:
# Verify results
print(weather_summary[['location', 'season', 
                       'thi_mean', 'thi_std', 'thi_95p', 'thi_5p',
                       'thi_mild_pct', 'thi_moderate_pct',
                       'thi_severe_pct', 'thi_high_stress_pct']].head())

  location  season   thi_mean   thi_std    thi_95p     thi_5p  thi_mild_pct  \
0   Bogura  2021S1  68.889424  5.842100  79.901313  59.663500      0.239583   
1   Bogura  2021S2  73.275808  5.946864  81.158700  61.287925      0.533333   
2   Bogura  2021S3  81.677443  2.488925  84.706400  76.910400      0.216495   
3   Bogura  2021S4  82.506543  1.325027  84.452550  80.083850      0.036585   
4   Bogura  2022S1  69.119698  4.722403  77.687125  63.152375      0.354167   

   thi_moderate_pct  thi_severe_pct  thi_high_stress_pct  
0          0.052083             0.0             0.052083  
1          0.088889             0.0             0.088889  
2          0.783505             0.0             0.783505  
3          0.963415             0.0             0.963415  
4          0.000000             0.0             0.000000  


In [71]:
print(weather_summary['thi_severe_pct'].tolist())

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [72]:
# Drop useless columns
weather_summary = weather_summary.drop(columns=['thi_severe_pct', 'thi_high_stress_pct'])

## Wind

In [74]:
# Check wind direction values
weather_full['wind_direction'].unique().tolist()

['South/South-easterly',
 'North/North-westerly',
 'West/North-westerly',
 'South/South-westerly',
 'North/North-easterly',
 'North//North-westerly',
 'North//North-easterly']

In [75]:
# Fix inconsistent notations
weather_full['wind_direction'] = weather_full['wind_direction'].replace({
    'North//North-westerly': 'North/North-westerly',
    'North//North-easterly': 'North/North-easterly'
})

weather_full['wind_direction'].unique().tolist()

['South/South-easterly',
 'North/North-westerly',
 'West/North-westerly',
 'South/South-westerly',
 'North/North-easterly']

In [76]:
# Simplify wind direction categories into abbreviations
wind_mapping = {
    'South/South-easterly': 'S_SE',
    'North/North-westerly': 'N_NW',
    'West/North-westerly': 'W_NW',
    'South/South-westerly': 'S_SW',
    'North/North-easterly': 'N_NE'
}

weather_full['wind_direction'] = weather_full['wind_direction'].map(wind_mapping)

weather_full['wind_direction'].unique().tolist()

['S_SE', 'N_NW', 'W_NW', 'S_SW', 'N_NE']

In [77]:
# Calculate wind speed statistics by location and season
wind_stats = weather_full.groupby(['location', 'season']).agg(
    wind_speed_mean=('wind_speed', 'mean'),
    wind_speed_std=('wind_speed', 'std'),
    total_days=('date', 'count')
).reset_index()

# Aggregate to weather_summary
weather_summary = weather_summary.merge(wind_stats, on=['location', 'season'])

In [78]:
# Calculate calm days percentage by group
calm_pct = (
    weather_full
    .assign(calm_day=lambda df: df['wind_speed'] < 2)
    .groupby(['location', 'season'])
    ['calm_day']
    .mean()  # Percentage = mean of boolean values
    .reset_index(name='calm_days_pct')
    .round(3)  # Round to 3 decimal places
)

# Merge into weather_summary
weather_summary = weather_summary.merge(
    calm_pct,
    on=['location', 'season'],
    how='left'
)

# Fill any missing values with 0
weather_summary['calm_days_pct'] = weather_summary['calm_days_pct'].fillna(0)

In [79]:
# Verify results
print(weather_summary[['location', 'season', 'calm_days_pct']].head())

  location  season  calm_days_pct
0   Bogura  2021S1          0.677
1   Bogura  2021S2          0.333
2   Bogura  2021S3          0.134
3   Bogura  2021S4          0.232
4   Bogura  2022S1          0.656


In [80]:
# Calculate wind direction proportions with suffix naming
directions = ['S_SE', 'N_NW', 'W_NW', 'S_SW', 'N_NE']

wind_pct = (
    weather_full.groupby(['location', 'season'])['wind_direction']
    .value_counts(normalize=True)
    .unstack(fill_value=0)
    .reindex(columns=directions, fill_value=0)
    .add_suffix('_pct')
    .reset_index()
)

# Merge with existing weather_summary
weather_summary = pd.merge(
    weather_summary,
    wind_pct,
    on=['location', 'season'],
    how='left'
)

# Fill any missing values with 0
for col in wind_pct.columns:
    if col.endswith('_pct') and col not in ['location', 'season']:
        weather_summary[col] = weather_summary[col].fillna(0)

In [81]:
# Verify results
print(weather_summary[['S_SE_pct', 'N_NW_pct', 'W_NW_pct', 'S_SW_pct', 'N_NE_pct']].head())

   S_SE_pct  N_NW_pct  W_NW_pct  S_SW_pct  N_NE_pct
0  0.135417  0.864583  0.000000  0.000000   0.00000
1  0.033333  0.522222  0.444444  0.000000   0.00000
2  0.000000  0.000000  0.350515  0.649485   0.00000
3  0.000000  0.000000  0.000000  1.000000   0.00000
4  0.000000  0.437500  0.000000  0.093750   0.46875


In [82]:
# Drop temperary column
weather_summary = weather_summary.drop(columns='total_days')

# Export the processed data frames

In [84]:
weather_full.to_csv('BMD_data_full_processed.csv', index=False)
weather_summary.to_csv('BMD_summary.csv', index=False)

In [85]:
# Generate a data dictionary
def create_data_dictionary(df, filename="BMD_data_data_dictionary.xlsx"):
    """
    Generate and export detailed data dictionary to Excel
    
    Parameters:
    df: Input DataFrame
    filename: Output Excel filename
    """
    # Create base dictionary
    data_dict = pd.DataFrame({
        'Variable Name': df.columns,
        'Data Type': df.dtypes.values,
        'Non-Null Count': df.count().values,
        'Description': [''] * len(df.columns),  # Placeholder for explanations
        'Source': ['BMD'] * len(df.columns)
    })
    
    # Add metadata
    data_dict['Unique Values'] = df.nunique().values
    
    # Initialize additional columns
    data_dict['Min Value'] = np.nan
    data_dict['Max Value'] = np.nan
    data_dict['Range'] = ''
    data_dict['Sample Values'] = ''
    
    # Process columns
    for idx, col in enumerate(df.columns):
        if pd.api.types.is_numeric_dtype(df[col]):
            min_val = df[col].min()
            max_val = df[col].max()
            data_dict.at[idx, 'Min Value'] = min_val
            data_dict.at[idx, 'Max Value'] = max_val
            data_dict.at[idx, 'Range'] = f"{min_val:.4f} to {max_val:.4f}"
        else:
            unique_vals = df[col].unique()
            sample = ', '.join(map(str, unique_vals[:min(3, len(unique_vals))]))
            if len(unique_vals) > 3:
                sample += ', ...'
            data_dict.at[idx, 'Sample Values'] = sample
    
    # Add automatic categorization
    data_dict['Category'] = '—'
    categories = {
        'radiation': ['PAR', 'UV'],
        'gwet': ['gwet'],
        'temperature': ['temp'],
        'gdd': ['gdd'],
        'thi': ['thi'],
        'humidity': ['rh', 'humidity'],
        'vpd': ['vpd'],
        'rainfall': ['rain', 'dry_days', 'wet_days'],
        'wind': ['wind', 'calm', 'SE', 'NW', 'SW', 'NE']
    }
    
    for idx, col_name in enumerate(df.columns):
        col_lower = col_name.lower()
        for category, keywords in categories.items():
            if any(kw.lower() in col_lower for kw in keywords):
                data_dict.at[idx, 'Category'] = category
                break
    
    # Reorder columns
    column_order = [
        'Variable Name', 'Category', 'Description', 'Data Type', 
        'Non-Null Count', 'Source', 'Unique Values', 
        'Min Value', 'Max Value', 'Range', 'Sample Values'
    ]
    data_dict = data_dict[column_order]
    
    # Export to Excel
    with pd.ExcelWriter(filename, engine='openpyxl') as writer:
        data_dict.to_excel(writer, index=False, sheet_name='Data Dictionary')
        
        # Add category legend to second sheet
        legend = pd.DataFrame({
            'Category': list(categories.keys()),
            'Keywords': [', '.join(kws) for kws in categories.values()]
        })
        legend.to_excel(writer, index=False, sheet_name='Category Legend')
    
    print(f"Data dictionary exported to: {filename}")
    return data_dict

# Usage
create_data_dictionary(weather_full)

/var/folders/qx/5zbpywt5661d6xr06v91bdyr0000gn/T/ipykernel_36799/3881315983.py:33: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'False' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  data_dict.at[idx, 'Min Value'] = min_val
/var/folders/qx/5zbpywt5661d6xr06v91bdyr0000gn/T/ipykernel_36799/3881315983.py:34: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'True' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  data_dict.at[idx, 'Max Value'] = max_val


Data dictionary exported to: BMD_data_data_dictionary.xlsx


,Variable Name,Category,Description,Data Type,Non-Null Count,Source,Unique Values,Min Value,Max Value,Range,Sample Values
0,location,—,,object,2190,BMD,3,NaN,NaN,,"Bogura, Cumilla, Mymensingh"
1,date,—,,datetime64[ns],2190,BMD,730,NaN,NaN,,"2020-10-22 00:00:00, 2020-10-23 00:00:00, 2020..."
2,rainfall,rainfall,,float64,2190,BMD,78,0.0,186.0,0.0000 to 186.0000,
3,temp_max_raw,temperature,,float64,2190,BMD,256,3.0,38.8,3.0000 to 38.8000,
4,temp_min_raw,temperature,,float64,2190,BMD,251,2.0,33.2,2.0000 to 33.2000,
5,rH_min_raw,humidity,,float64,2190,BMD,140,16.756043,97.0,16.7560 to 97.0000,
6,rH_max_raw,humidity,,float64,2190,BMD,94,63.0,100.0,63.0000 to 100.0000,
7,wind_speed_raw,wind,,float64,2190,BMD,144,0.0,25.0,0.0000 to 25.0000,
8,wind_direction,wind,,object,2190,BMD,5,NaN,NaN,,"S_SE, N_NW, W_NW, ..."
9,season,wind,,object,2190,BMD,8,NaN,NaN,,"2021S1, 2021S2, 2021S3, ..."
